In [8]:
import numpy as np
import pandas as pd


In [9]:
inputpath='../../trends_data/preprocess/pureSBER315.csv'
historyOutPath='../../trends_data/preprocess/shit.csv'


In [10]:
columns = ['<TIME>', '<VOLUME>', '<PRICE>']
###############################################################################
data = pd.read_csv(inputpath, sep = ',', names=columns, header = 0)
y_column = data['<PRICE>']
y_array = np.array(y_column)
x_list = np.array(range(len(y_array)))
ticks_total = len(y_array) - 1

step_read = 60
Lmax = 400000
zazor_coeff = 0.00005
relax_coef = 1
# сетка на 800 делений
number_of_bars = 800

# место остановки окна
pitstop = ticks_total - 1000
min_trend_len = 3000

# старт движения окна
breaker = 5000

# инициализация переменной не нулем и не единицей
direction = 456
x_stop = 0

In [33]:
def get_delimeters(breaker, number_of_bars, window_max_size):
    """
    :param breaker: end tick of the window
    :param number_of_bars: number of bars in
        window separating grid
    :return: a numpy array of ticks corresponding to
        hyperbolic grid for a given window
    """

    if breaker <= window_max_size:
        window_starting_tick = 10
        window_size = breaker - window_starting_tick
    else:
        window_starting_tick = breaker - window_max_size
        window_size = window_max_size

    k = window_size/np.log(number_of_bars)
    delimiters_list = []
    for i in range(1,number_of_bars+1):
        window_starting_tick += int(k/i)
        delimiters_list.append(window_starting_tick)

    return np.array(delimiters_list)


def get_grid_extremum_coordinates(y_array, window_delimiters):
    """
    takes arrays of ticks and corresponding prices, cuts a
    window corresponding to window's_delimeters boundaries.
    Slices this 2D array according to  window_delimeters list.

    :return: dataframe with 2D arrays of ticks and prices
        for each grid section and section number as index
    """
    #TODO: check the code
    #нарезаем на куски y согласно разделителям
    sliced_y = [y_array[window_delimiters[i]:window_delimiters[i+1]] for i in range(len(window_delimiters)-1)]
    #записываем координаты x соответствующие началу каждого куска
    x_start_of_slice = window_delimiters[:-1]
    #собираем в датафрейм
    #TODO: column name change will break append_extremums function. Bad design
    slice_dict = {'y_slice': sliced_y,
                  'x_start_of_slice': x_start_of_slice}
    sliced_window_df = pd.DataFrame.from_dict(slice_dict)

    return sliced_window_df


# noinspection PyUnreachableCode
def append_extremums(sliced_window_df):
    """
    takes sliced_window_df and calculates min and max
    coordinates for each window section
    :param sliced_window_df:
    :return: df with coordinates of max and mins
    """
    #TODO: check the code
    sliced_window_df['max_y'] = sliced_window_df['y_slice'].apply(np.max)
    sliced_window_df['max_x'] = sliced_window_df['y_slice'].apply(np.argmax) + sliced_window_df['x_start_of_slice']
    sliced_window_df['min_y'] = sliced_window_df['y_slice'].apply(np.min)
    sliced_window_df['min_x'] = sliced_window_df['y_slice'].apply(np.argmin) + sliced_window_df['x_start_of_slice']

    return sliced_window_df



In [26]:
delitemers = get_delimeters(500000, 800, Lmax)

In [34]:
sliced_window_df = get_grid_extremum_coordinates(y_array, delitemers)

In [35]:
sliced_window_df.head()

,y_slice,x_start_of_slice
0,"[5867.0, 5867.0, 5867.0, 5863.0, 5863.0, 5866....",159838
1,"[5939.0, 5936.0, 5936.0, 5939.0, 5937.0, 5937....",189757
2,"[6026.0, 6025.0, 6025.0, 6030.0, 6029.0, 6026....",209703
3,"[6114.0, 6114.0, 6112.0, 6115.0, 6115.0, 6115....",224662
4,"[6367.0, 6368.0, 6369.0, 6369.0, 6369.0, 6369....",236629


In [37]:
exrems = append_extremums(sliced_window_df)

In [42]:
exrems.iloc[-1]['x_start_of_slice']

534081

In [41]:
exrems.tail()

,y_slice,x_start_of_slice,max_y,max_x,min_y,min_x
794,"[6487.0, 6481.0, 6480.0, 6486.0, 6485.0, 6484....",533783,6487.0,533783,6478.0,533854
795,"[6480.0, 6479.0, 6479.0, 6480.0, 6481.0, 6480....",533858,6486.0,533898,6475.0,533931
796,"[6474.0, 6474.0, 6473.0, 6473.0, 6473.0, 6473....",533933,6489.0,533999,6472.0,533939
797,"[6488.0, 6489.0, 6489.0, 6484.0, 6489.0, 6489....",534007,6493.0,534033,6483.0,534063
798,"[6487.0, 6486.0, 6486.0, 6491.0, 6487.0, 6487....",534081,6499.0,534117,6482.0,534150


In [43]:
working_max_y = exrems.iloc[-1]['max_y']
working_min_y = exrems.iloc[-1]['max_x']
working_max_x = exrems.iloc[-1]['min_y']
working_min_x = exrems.iloc[-1]['min_x']

In [44]:
exrems['max_k'] = (working_max_y - exrems['max_y'])/(working_max_x - exrems['min_x'])

In [45]:
exrems.tail()

,y_slice,x_start_of_slice,max_y,max_x,min_y,min_x,max_k
794,"[6487.0, 6481.0, 6480.0, 6486.0, 6485.0, 6484....",533783,6487.0,533783,6478.0,533854,-0.000023
795,"[6480.0, 6479.0, 6479.0, 6480.0, 6481.0, 6480....",533858,6486.0,533898,6475.0,533931,-0.000025
796,"[6474.0, 6474.0, 6473.0, 6473.0, 6473.0, 6473....",533933,6489.0,533999,6472.0,533939,-0.000019
797,"[6488.0, 6489.0, 6489.0, 6484.0, 6489.0, 6489....",534007,6493.0,534033,6483.0,534063,-0.000011
798,"[6487.0, 6486.0, 6486.0, 6491.0, 6487.0, 6487....",534081,6499.0,534117,6482.0,534150,-0.000000


In [49]:
def append_trend_equations(df):
    """
    adds 4 columns with trend's equation parameters for
    mins and maxs: min_k, min_b, max_k, max_b

    Equation of line connecting current extremum
    with the last extremum of the window

    :param sliced_window_extremums_df:
    :return: df with 4 added columns of coefficients
    """
    working_max_y = df.iloc[-1]['max_y']
    working_min_y = df.iloc[-1]['max_x']
    working_max_x = df.iloc[-1]['min_y']
    working_min_x = df.iloc[-1]['min_x']
    df['max_k'] = (working_max_y - df['max_y']) / (working_max_x - df['max_x'])
    df['max_b'] = working_max_y - df['max_k']*working_max_x
    df['min_k'] = (working_min_y - df['min_y']) / (working_min_x - df['min_x'])
    df['min_b'] = working_min_x - df['min_k']*working_min_x

    #TODO: code

    return df

In [50]:
append_trend_equations(exrems).tail()

,y_slice,x_start_of_slice,max_y,max_x,min_y,min_x,max_k,max_b,min_k,min_b
794,"[6487.0, 6481.0, 6480.0, 6486.0, 6485.0, 6484....",533783,6487.0,533783,6478.0,533854,-0.000023,6499.147513,1782.564189,-9.516225e+08
795,"[6480.0, 6479.0, 6479.0, 6480.0, 6481.0, 6480....",533858,6486.0,533898,6475.0,533931,-0.000025,6499.159771,2409.324201,-1.286406e+09
796,"[6474.0, 6474.0, 6473.0, 6473.0, 6473.0, 6473....",533933,6489.0,533999,6472.0,533939,-0.000019,6499.122878,2500.687204,-1.335208e+09
797,"[6488.0, 6489.0, 6489.0, 6484.0, 6489.0, 6489....",534007,6493.0,534033,6483.0,534063,-0.000011,6499.073722,6064.758621,-3.238957e+09
798,"[6487.0, 6486.0, 6486.0, 6491.0, 6487.0, 6487....",534081,6499.0,534117,6482.0,534150,-0.000000,6499.000000,inf,-inf
